<h1><center>  对比分析 -  Nucleus API用例</center></h1>


<h1><center>  所有权及保密条款属 SumUp Analytics 所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取</center></h1>


 


## 目标 : 
-	开发一个对比分析来定制和微调数据集
  - 提取对比的主题
  - 对比总结
  - 将文档分为2个预定义类别

**SumUp对比分析的原理是一个语库中，由用户根据元数据或内容自定义的两种不同类别的文档进行对比分析**

## 数据:
-	来自同一领域的任何文档集合，有进一步细分的可能行，比如说，还可以根据文档类型等分类

    **Nucleus Datafeed可直接调用主要中央银行和SEC文件的所有内容**


## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	对比主题建模 API
 - 	*api_instance.post_topic_contrast_api(payload)*
 
 
-	文件对比摘要 API
 - 	*api_instance.post_document_contrast_summary_api(payload)*


-	文件分类 API
 - 	*api_instance.post_doc_classify_api(payload)*

## Approach:

### 1.	Dataset Preparation
-	Create a Nucleus dataset containing all relevant documents

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
print('---- Case 1: you are using your own corpus, coming from a local folder ----')
folder = 'Sellside_research'         
dataset = 'Sellside_research'# str | Destination dataset where the file will be inserted.

# 以递归方式从文件夹构建文件。
# 其中每个项目采用以下格式：
# {'filename': filename,   # 要上传的文件名。 必填
#  'metadata': {           # 该文件的元数据。 选填
#      'key1': val1,       # 密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'company': 'Apple',
                                      'research_analyst': 'MS',
                                      'date': '2019-01-01'}}
            file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

    
    
print('---- Case 2: you are using an embedded datafeed ----')
dataset = 'sumup/central_banks_chinese'# Nucleus自带数据库。
metadata_selection = {'bank': 'people_bank_of_china', 'document_category': ('speech', 'press release')}

### 2. Contrasted Topic Modeling

-     In this example, we define one category of documents as being produced by research analysts at Morgan Stanley. The second category of documents will be comprised of all other research reports.
-     We extract one topic that separates those two categories

In [ ]:
metadata_selection_contrast = {"research_analyst": "MS"} # dict | 元数据选择定义了两类文档，以便相互对比和总结

query = '' # str | Dataset-language-specific的全文查询，使用mysql MATCH boolean query 格式（可选）
custom_stop_words = ["morgan stanley"] # 停用词列表。 （可选）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 指定是否考虑每类文档的语法方面以帮助对比它们（可选）（默认为False）
num_keywords = 20 # integer | 从数据集中提取的比较主题的关键字数。 (可选) (默认为50)
remove_redundancies = False # bool | 如果为True，则此选项从分析中删除准重复项，并仅重新获取其副本。 准复制将用具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）

payload = nucleus_api.TopicContrastModel(dataset='Sellside_research', 
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        period_start='2018-01-01',
                                        period_end='2019-01-01')
try:
    api_response = api_instance.post_topic_contrast_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    print('Contrasted Topic')
    print('    Keywords:', api_response.result.keywords)
    print('    Keywords Weight:', api_response.result.keywords_weight)

### 3.	文件对比摘要

-   使用相同的数据集，我们的目标是找到将文档分为两类的摘要句子


-	使用以下输入参数来控制摘要的大小并过滤过短或过长的句子。
    - `summary_length`
    - `context_amount` (每个关键摘要句子周围的句子数)
    - `short_sentence_length`
    - `long_sentence_length`
    

-	设置以下参数以调整或细化对比摘要的焦点和内容
    - `custom_stop_words` (自定义停用词列表)
    - `syntax_variables` (包括/排除语法变量)
    - `num_keywords` (控制对比度总结的宽度)
    - `remove_redundancies` (删除冗余)


-	接下来，我们将讨论如何构建自定义的停用词列表以优化文档对比摘要

In [ ]:
print('---------------- Get doc contrasted summaries ------------------------')
metadata_selection_contrast = {"research_analyst": "MS"} # dict | 元数据选择定义了两类文档，以便相互对比和总结

query = '' # str | 数据集语言特定的全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = ["morgan stanley"] # 停用词列表。 （可选的）
summary_length = 6 # int | 用户希望在对比摘要中看到的最大点句。 （可选）（默认为6）
context_amount = 0 # int | 与点句相关的句子数量。 （可选）（默认为0）
short_sentence_length = 0 # int | 排除的短句子长度（可选）（默认为4）
long_sentence_length = 40 # int | 排除的长句子长度（可选） （默认为40）
excluded_docs = '' # str | 排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = True # bool | 在进行对比分析是，是否考虑文档的语法来帮助对比（可选）（默认为False）
num_keywords = 20 # integer | 从数据集中提取的比较主题的关键字数。 (可选) (默认为50)
remove_redundancies = False # bool | 如果为True，则此选项从分析中删除准重复项，并仅重新获取其副本。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）

payload = nucleus_api.DocumentContrastSummaryModel(dataset="Sellside_research", 
                                                    metadata_selection_contrast=metadata_selection_contrast,
                                                    custom_stop_words=custom_stop_words,
                                                    period_start='2018-01-01',
                                                    period_end='2019-01-01')
try:
    api_response = api_instance.post_document_contrast_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    print('Summary for', [x for x in  metadata_selection_contrast.values()])
    for sent in api_response.result.class_1_content.sentences:
        print('    *', sent)
    print('======')
    for sent in api_response.result.class_2_content.sentences:
        print('    *', sent)   

### 4. 文件分类

此任务需要3个步骤:
-    在标记的数据集上提取对比度主题
-    通过提供标记数据集来训练具有对比度主题的分类器。 在此步骤中，您可以从对比主题调整每个关键字的权重，删除某些关键字，甚至将步骤1生成的对比主题与您自己选择的主题进行比较
-    使用测试集测试分类器

-    在下面的示例中，我们假设已经获得了对比主题。 'fixed_topics'的结构正是Contrasted Topic API的结构

In [ ]:
fixed_topics = {"keywords": ["price target", "projected revenue", "economy"], "weights": [0.5, 0.25, 0.25]} # dict | 对比主题用于分隔两类文件。 重量可自选
metadata_selection_contrast = {"research_analyst": "MS"} # dict | 元数据选择定义了文档可以分类的两类文档

query = '' # str | 数据集语言特定的全文查询，使用mysql MATCH boolean格式（可选）
custom_stop_words = ["morgan stanley"] # 停用词列表。 （可选的）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = True # bool | 如果为True，则分类器将在内容变量之上包含与语法相关的变量（可选）（默认为False）
threshold = 0 # float | 文档暴露于对话主题的阈值，在该阈值之上，文档被分配给通过metadata_selection指定的类1。 （可选）（默认为0）
remove_redundancies = False # bool | 如果为True，则此选项从分析中删除准重复项，并仅重新获取其副本。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）

payload = nucleus_api.DocClassifyModel(dataset="Sellside_research",
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        validation_phase=True,
                                        period_start='2018-01-01',
                                        period_end='2019-01-01')
try:
    api_response = api_instance.post_doc_classify_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    print('Detailed Results')
    print('    Docids:', api_response.result.detailed_results.docids)
    print('    Exposure:', api_response.result.detailed_results.exposures)
    print('    Estimated Category:', api_response.result.detailed_results.estimated_class)
    print('    Actual Category:', api_response.result.detailed_results.true_class)
    print('\n')

    print('Perf Metrics')
    print('    Accuracy:', api_response.result.perf_metrics.hit_rate)
    print('    Recall:', api_response.result.perf_metrics.recall)
    print('    Precision:', api_response.result.perf_metrics.precision)

接下来，我们可以进入测试阶段

In [ ]:
fixed_topics = {"keywords": ["price target", "projected revenue", "economy"], "weights": [0.5, 0.25, 0.25]} # dict | 对比主题用于分隔两类文件
metadata_selection_contrast = {"research_analyst": "MS"} # dict | 元数据选择定义了文档可以分类的两类文档

payload = nucleus_api.DocClassifyModel(dataset="Sellside_research",
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        validation_phase=False,
                                        period_start='2019-01-02',
                                        period_end='2019-06-01')
try:
    api_response = api_instance.post_doc_classify_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    print('Detailed Results')
    print('    Docids:', api_response.result.detailed_results.docids)
    print('    Exposure:', api_response.result.detailed_results.exposures)
    print('    Estimated Category:', api_response.result.detailed_results.estimated_class)

### 5.	微调

#### a.	从对比度分析中排除某些内容

-   对比度分析API中,通过使用`custom_stop_words`参数，排除无关的关键字/主题以定制对比度分析


-	在语料库中的文档中提取对比主题的关键字，并打印这些主题的关键字



In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Sellside_research',                         
                            query='',                       
                            num_topics=8, 
                            num_keywords=8,
                            metadata_selection=metadata_selection_contrast)
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

根据您的专业知识和经验，可以确定特定主题或关键字是否具备足够的差异来促进对比度分析。

然后，您可以通过创建包含这些单词的`custom_stop_words`变量来定制对比度分析。 如下所示，初始化变量并将其传递到第2节主代码的有效负载中：

In [1]:
custom_stop_words = ["disclaimer","disclosure"] # str | 停用词列表。 （可选）

#### b. 将对比摘要集中在您的语料库中可能讨论的特定主题上
**query**: 您可以通过利用Doc Contrasted Summary API的查询变量来优化对比度分析.

使用特定查询优化，重新运行Contrast Analysis APIs(对比度分析APIs)。 创建变量并将其传递给有效载荷（payload)：

In [ ]:
query = '(earnings OR cash flows)' # str | 全文查询，使用mysql MATCH boolean query格式。 例如：“（word1或word2）AND（word3或word4）”（可选）

#### c. 对您的对比主题指定`metadata_selection_contrast`

-     对比两个不同实体的文件

    在您自己的数据上，例如，卖方研究：

In [ ]:
metadata_selection_contrast = {"research_analyst": ["MS", "JPM"]}

    在SumUp自带数据上，例如， 中央银行：

In [ ]:
metadata_selection_contrast = {"bank": ["federal_reserve", "ECB"]}

-     对比给定实体的不同文档

    在SumUp自带数据上，例如， 中央银行：

In [ ]:
metadata_selection_contrast = {"document_category": ["speech", "press release"]}

-     对比包含不同关键字的文档

    在您自己的数据上，或在SumUp数据Feed上： 

In [ ]:
metadata_selection_contrast = {"content": "fundamentals"}

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 